In [3]:
# uncomment and run below:
%pip install -qU langchain
%pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 1.3 MB/s eta 0:00:0031m1.2 MB/s eta 0:00:01
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.77
    Uninstalling langsmith-0.1.77:
      Successfully uninstalled langsmith-0.1.77


# Introduction to LangChain 

Working with LLMs involves in one way or another working with a specific type of abstraction: "Prompts".

However, in the practical context of day-to-day tasks we expect LLMs to perform, these prompts won't be some static and dead type of abstraction. Instead we'll work with dynamic prompts re-usable prompts.

# Lanchain

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework that allows you to connect LLMs together by allowing you to work with modular components like prompt templates and chains giving you immense flexibility in creating tailored solutions powered by the capabilities of large language models.


Its main features are:
- **Components**: abstractions for working with LMs
- **Off-the-shelf chains**: assembly of components for accomplishing certain higher-level tasks

LangChain facilitates the creation of complex pipelines that leverage the connection of components like chains, prompt templates, output parsers and others to compose intricate pipelines that give you everything you need to solve a wide variety of tasks.

At the core of LangChain, we have the following elements:

- Models
- Prompts
- Output parsers

**Models**

Models are nothing more than abstractions over the LLM APIs like the ChatGPT API.​

In [31]:
import os
import getpass

# # Set OPENAI API Key

os.environ["OPENAI_API_KEY"] = getpass.getpass()
# OR (load from .env file)
# make sure you have python-dotenv installed
# from dotenv import load_dotenv
# load_dotenv("./.env")

In [1]:
from langchain_openai import ChatOpenAI

In [2]:
MODEL='gpt-4o-mini'

In [3]:
chat_model = ChatOpenAI(model=MODEL, temperature=0)

In [4]:
output = chat_model.invoke("I am teaching a live-training about LLMs!")
output

AIMessage(content='That sounds exciting! Large Language Models (LLMs) are a fascinating topic with a lot of depth. What specific aspects are you planning to cover in your training? Here are some ideas you might consider including:\n\n1. **Introduction to LLMs**: Explain what LLMs are, how they work, and their significance in the field of AI.\n\n2. **Architecture**: Discuss the architecture of popular LLMs like GPT, BERT, and others. You could cover concepts like transformers, attention mechanisms, and tokenization.\n\n3. **Training Process**: Explain how LLMs are trained, including the data used, the training process, and the challenges involved.\n\n4. **Applications**: Highlight various applications of LLMs, such as chatbots, content generation, translation, and more.\n\n5. **Ethical Considerations**: Discuss the ethical implications of using LLMs, including bias, misinformation, and the environmental impact of training large models.\n\n6. **Hands-On Demonstration**: If possible, incl

In [5]:
type(output)

langchain_core.messages.ai.AIMessage

In [6]:
print(output.content)

That sounds exciting! Large Language Models (LLMs) are a fascinating topic with a lot of depth. What specific aspects are you planning to cover in your training? Here are some ideas you might consider including:

1. **Introduction to LLMs**: Explain what LLMs are, how they work, and their significance in the field of AI.

2. **Architecture**: Discuss the architecture of popular LLMs like GPT, BERT, and others. You could cover concepts like transformers, attention mechanisms, and tokenization.

3. **Training Process**: Explain how LLMs are trained, including the data used, the training process, and the challenges involved.

4. **Applications**: Highlight various applications of LLMs, such as chatbots, content generation, translation, and more.

5. **Ethical Considerations**: Discuss the ethical implications of using LLMs, including bias, misinformation, and the environmental impact of training large models.

6. **Hands-On Demonstration**: If possible, include a live demo where participa

You can predict outputs from both LLMs and ChatModels:

Basic components are:

- Models
- Prompt templates
- Output parsers

In [7]:
from langchain_core.prompts import ChatPromptTemplate

In [8]:
template = "Show me 5 examples of this concept: {concept}"

prompt = ChatPromptTemplate.from_template(template)

prompt.format(concept="animal")

'Human: Show me 5 examples of this concept: animal'

In [9]:
chain = prompt | chat_model

In [10]:
type(chain)

langchain_core.runnables.base.RunnableSequence

In [11]:
output = chain.invoke({"concept": "animal"})

In [12]:
output.content

'Sure! Here are five examples of different types of animals:\n\n1. **Mammal**: **Elephant** - The largest land animal, known for its intelligence, social behavior, and strong familial bonds.\n\n2. **Bird**: **Bald Eagle** - A bird of prey known for its white head and tail, symbolizing strength and freedom, and is the national bird of the United States.\n\n3. **Reptile**: **Green Iguana** - A large lizard native to Central and South America, known for its vibrant green color and ability to adapt to various environments.\n\n4. **Amphibian**: **Poison Dart Frog** - A small, brightly colored frog found in Central and South America, known for its toxic skin that can be lethal to predators.\n\n5. **Fish**: **Clownfish** - A small, colorful fish that lives in sea anemones, known for its symbiotic relationship with the anemone and popularized by the movie "Finding Nemo."\n\nThese examples illustrate the diversity of the animal kingdom across different classes.'

In [13]:
from IPython.display import Markdown


Markdown(output.content)

Sure! Here are five examples of different types of animals:

1. **Mammal**: **Elephant** - The largest land animal, known for its intelligence, social behavior, and strong familial bonds.

2. **Bird**: **Bald Eagle** - A bird of prey known for its white head and tail, symbolizing strength and freedom, and is the national bird of the United States.

3. **Reptile**: **Green Iguana** - A large lizard native to Central and South America, known for its vibrant green color and ability to adapt to various environments.

4. **Amphibian**: **Poison Dart Frog** - A small, brightly colored frog found in Central and South America, known for its toxic skin that can be lethal to predators.

5. **Fish**: **Clownfish** - A small, colorful fish that lives in sea anemones, known for its symbiotic relationship with the anemone and popularized by the movie "Finding Nemo."

These examples illustrate the diversity of the animal kingdom across different classes.

You can also use the predict method over a string input:

In [18]:
text = "What would be a good name for a dog that loves to nap??"
chat_model.invoke(text)

AIMessage(content="Here are some cute name ideas for a dog that loves to nap:\n\n1. Snoozer\n2. Naptime\n3. Dreamer\n4. Dozer\n5. Snuggles\n6. Siesta\n7. Zzz\n8. Pillow\n9. Napster\n10. Drowse\n\nChoose one that fits your dog's personality!", response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 21, 'total_tokens': 95}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-e7347bfd-1c7f-4c07-b88c-547be07ce054-0', usage_metadata={'input_tokens': 21, 'output_tokens': 74, 'total_tokens': 95})

**Prompts**

The same works for prompts. Now, prompts are pieces of text we feed to LLMs, and LangChain allows you to work with prompt templates.

Prompt Templates are useful abstractions for reusing prompts and they are used to provide context for the specific task that the language model needs to complete. 

A simple example is a `PromptTemplate` that formats a string into a prompt:

In [14]:
from langchain_core.prompts  import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")
prompt.format(activity="sleeping")
# Output: "What is a good dog name for a dog that loves to nap?"

'Human: What is a good dog name for a dog that loves to sleeping?'

In [15]:
chain = prompt | chat_model

chain.invoke({'activity': 'sleeping'})

AIMessage(content="Here are some cute and fitting names for a dog that loves to sleep:\n\n1. **Snoozer**\n2. **Napster**\n3. **Dozer**\n4. **Slumber**\n5. **Dreamer**\n6. **Pillow**\n7. **Cuddles**\n8. **Resty**\n9. **Zzz**\n10. **Naptime**\n\nChoose one that resonates with your dog's personality!", response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 21, 'total_tokens': 109}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-8ef93ae2-9ec7-497d-a55f-ece436cf658e-0', usage_metadata={'input_tokens': 21, 'output_tokens': 88, 'total_tokens': 109})

**Output Parsers**

OutputParsers convert the raw output from an LLM into a format that can be used downstream. Here is an example of an OutputParser that converts a comma-separated list into a list:

In [20]:
chain.invoke({"concept": "Landscapes"})

AIMessage(content='Sure! Here are five examples of different types of landscapes:\n\n1. **Mountain Landscape**: A breathtaking view of the Rocky Mountains, featuring towering peaks, lush green valleys, and a clear blue sky. This landscape often includes elements like alpine lakes, dense forests, and wildlife such as deer and eagles.\n\n2. **Desert Landscape**: A vast expanse of the Sahara Desert, characterized by rolling sand dunes, sparse vegetation, and a dramatic sunset that casts warm hues across the horizon. This landscape may also include unique rock formations and occasional oases.\n\n3. **Coastal Landscape**: A picturesque scene of the Amalfi Coast in Italy, showcasing steep cliffs adorned with colorful villages, crystal-clear waters, and lush Mediterranean vegetation. The landscape is often dotted with boats and features stunning views of the coastline.\n\n4. **Forest Landscape**: A serene view of a temperate rainforest, such as the Pacific Northwest, filled with towering tree

In [16]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [17]:
llm = ChatOpenAI(model=MODEL, temperature=0.0)
prompt = ChatPromptTemplate.from_template("""
Write 5 concepts that are fundamental to learn about {topic}.
                                          """)
chain = prompt | llm | output_parser
chain.invoke({"topic": "Artificial Neural Networks"})

"Certainly! Here are five fundamental concepts that are essential to understand when learning about Artificial Neural Networks (ANNs):\n\n1. **Neurons and Activation Functions**:\n   - At the core of ANNs are artificial neurons, which are inspired by biological neurons. Each neuron receives inputs, processes them, and produces an output. The output is typically passed through an activation function, which introduces non-linearity into the model. Common activation functions include Sigmoid, ReLU (Rectified Linear Unit), and Tanh. Understanding how these functions work and their impact on the network's performance is crucial.\n\n2. **Network Architecture**:\n   - The architecture of an ANN refers to the arrangement of neurons in layers, including the input layer, hidden layers, and output layer. The number of layers and the number of neurons in each layer can significantly affect the network's ability to learn and generalize. Concepts such as feedforward networks, convolutional neural ne

This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an output parser.

Ok, so these are the basics of langchain. But how can we leverage these abstraction capabilities inside our LLM app application?

Now, to put everything together LangChain allows you to build something called "chains", which are components that connect prompts, llms and output parsers into a building block that allows you to create more interesting and complex functionality.

Let's look at the example below:

So, what the chain is doing is connecting these basic components (the LLM and the prompt template) into
a block that can be run separately. The chain allows you to turn workflows using LLLMs into this modular process of composing components.

Now, the newer versions of LangChain have a new representation language to create these chains (and more) known as LCEL or LangChain expression language, which is a declarative way to easily compose chains together. The same example as above expressed in this LCEL format would be:

In [24]:
chain = prompt | llm

chain.invoke({"topic": "sleep"})

AIMessage(content='Here are five fundamental concepts to learn about sleep:\n\n1. **Sleep Stages and Cycles**: Sleep is divided into several stages, including Non-REM (Rapid Eye Movement) and REM sleep. Non-REM sleep further consists of three stages (N1, N2, and N3), with N3 being deep sleep. Understanding these stages helps elucidate the restorative processes that occur during sleep, such as physical repair, memory consolidation, and emotional regulation.\n\n2. **Circadian Rhythms**: Circadian rhythms are the body’s internal clock, regulating the sleep-wake cycle roughly every 24 hours. Factors such as light exposure, hormone levels (like melatonin), and lifestyle choices influence these rhythms. Disruptions to circadian rhythms can lead to sleep disorders and negatively impact overall health.\n\n3. **Sleep Hygiene**: Sleep hygiene refers to a set of practices and habits that promote good quality sleep. This includes maintaining a consistent sleep schedule, creating a conducive sleep 

In [18]:
from langchain_ollama import ChatOllama

In [19]:
# ollama pull llama3 in the terminal
llm = ChatOllama(model="llama3.1")

In [21]:
type(llm)

langchain_ollama.chat_models.ChatOllama

In [22]:
chain = prompt | llm | output_parser

chain.invoke({"topic": "neuroscience of sleep"})

"Here are five key concepts that are fundamental to learning about the neuroscience of sleep:\n\n1. **Sleep-Wake Cycle Regulation**: The suprachiasmatic nucleus (SCN) is the master biological clock that regulates the body's 24-hour sleep-wake cycle, also known as the circadian rhythm. This cycle involves a complex interplay between light exposure, hormones, and neural activity to maintain our natural sleep-wake patterns.\n\n2. **Sleep Stage Architecture**: Sleep is not just one continuous process but rather a series of different stages that occur in a cyclical pattern throughout the night. These stages include:\n\t* NREM (non-rapid eye movement) sleep: divided into three sub-stages (N1, N2, and N3), characterized by decreasing cortical activity and increasing parasympathetic dominance.\n\t* REM (rapid eye movement) sleep: marked by rapid eye movements, low muscle tone, increased brain activity, and vivid dreams.\n\n3. **Neurotransmitters and Sleep**: Various neurotransmitters play a cr

Notice that now the output is an `AIMessage()` object, which represents LangChain's way to abstract the output from an LLM model like ChatGPT or others.

These building blocks and abstractions that LangChain provides are what makes this library so unique, because it gives you the tools you didn't know you need it to build awesome stuff powered by LLMs.

# LangChain Exercise

Let's create a simple chain for summarization of content. 

Your chain should:

- A prompt template with one or more variables
- A model like ChatGPT or other (you can use local models if you'd like, I recommend `ChatOllama` for that!)
- Optional: use output parsing or just fetch the string output at the end!

## Example Answer

Let's make use of the `ChatPromptTemplate` to abstract away the following pieces of the prompt: 
- `content` - the text content to be summarized  
- `summary_format` - the format in which we want the summary to be presented (like bullet points and so on).

In [21]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Summarize this: {content}. The output should be in the following format: {summary_format}.")

# We can look at a simple example to illustrate what that prompt is doing
prompt.format(content="This is a test.", summary_format="One word summary")

'Human: Summarize this: This is a test.. The output should be in the following format: One word summary.'

Ok, now that we have our prompt template done, let's load the llm and create a nice chain to put everything together. 

In [22]:
from langchain_openai import ChatOpenAI

llm_chat =  ChatOpenAI()
chain = prompt | llm_chat # This is the Pipe symbol! from LCEL that connect model to prompt!

Now, that we have our chain we can run some tests. The cool thing about working with LLMs is that you can use them to create examples for simple tests like this (avoiding the annoynace of searching online for some piece of text, copying and pasting etc...). So, let's generate a few examples of tests below:

In [23]:
num_examples = 3
examples = []
for i in range(num_examples):
    examples.append(llm_chat.invoke("Create a piece of text with 2 paragraphs about a random topic regarding human-machine interaction."))

examples

[AIMessage(content='As technology continues to advance, the interaction between humans and machines becomes more integrated into everyday life. From virtual assistants like Siri and Alexa to self-driving cars, the reliance on machines to assist in our daily tasks is only growing. This shift in human-machine interaction raises questions about the future of work and the role of automation in society.\n\nWhile machines can greatly improve efficiency and productivity, there are concerns about the potential impact on employment. As more tasks become automated, there is a fear that jobs will be displaced and workers will be left without opportunities. However, proponents of automation argue that it can create new job opportunities and allow humans to focus on more creative and strategic tasks. Finding a balance between human labor and machine automation will be crucial in navigating the future of work and ensuring that both humans and machines can coexist harmoniously.', response_metadata={'

Nice! Now that we have our examples, let's run our chain on them and check out the results.

In [24]:
summary_format = "bullet points"

outputs = []
for ex in examples:
    outputs.append(chain.invoke({"content": ex, "summary_format": summary_format}))

# Let's display one example output
outputs[0]

AIMessage(content='- Technology advances lead to increased integration of humans and machines in daily life\n- From virtual assistants to self-driving cars, reliance on machines for daily tasks is growing\n- Shift raises questions about the future of work and role of automation in society\n- Concerns about potential impact on employment as tasks become automated\n- Debate between job displacement and creation of new opportunities through automation\n- Finding a balance between human labor and machine automation is crucial for harmonious coexistence.', response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 310, 'total_tokens': 403}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-67ec51b2-de36-4ead-ba88-af5d76418cf1-0', usage_metadata={'input_tokens': 310, 'output_tokens': 93, 'total_tokens': 403})

Great! So it seems our chain worked and we generated some summaries! Let's visualize all the summaries generated in a neat way.

In [25]:
from IPython.display import Markdown

for i in range(num_examples):
    display(Markdown(f"Output {i} \n {outputs[i].content}"))
# Markdown(f"**Input**: {examples[0]}\n\n**Output**: {outputs[0]}")

Output 0 
 - Technology advances lead to increased integration of humans and machines in daily life
- From virtual assistants to self-driving cars, reliance on machines for daily tasks is growing
- Shift raises questions about the future of work and role of automation in society
- Concerns about potential impact on employment as tasks become automated
- Debate between job displacement and creation of new opportunities through automation
- Finding a balance between human labor and machine automation is crucial for harmonious coexistence.

Output 1 
 - As technology advances, humans and machines are becoming more intertwined in various aspects of daily life.
- The reliance on machines for communication and problem-solving has raised concerns about privacy and security.
- Artificial intelligence is rapidly growing, with AI systems becoming more sophisticated in understanding and responding to human language.
- Ethical debates surrounding AI focus on its potential impact on jobs, decision-making processes, and societal values.
- It is important to consider the implications of our reliance on machines in shaping future interactions.

Output 2 
 - Technology is advancing, making interactions between humans and machines more seamless
- Voice-activated virtual assistants and self-driving cars are examples of evolving interactions with machines
- Concerns arise about the impact on daily lives and the potential for machines to replace human tasks
- Job displacement is a key concern as machines become more capable
- Proponents argue that human-machine interaction can lead to greater efficiency and productivity
- Finding a balance between harnessing machine capabilities and preserving human skills is crucial for successful interaction.

Great! Our summaries worked, and we were able to apply a given summary format to all of them.

LangChain is an extremely powerful library to work with abstractions like these and throughout this course we hope to give you a gliimpse of the cool stuff you can build with it.